# Clustering- Finding Related Posts

## Preprocessing -  similarity measured as similar number of common words

We do not have to write a custom code for counting words and representing those counts as vector. Scikit's CountVectorizer does the job very efficiently. It also has a very convenient interface. 

In [2]:
from sklearn.feature_extraction.text import CountVectorizer
# parameter: minimun document frequency, all words ocurring less that that value
# will be dropped.
vectorizer = CountVectorizer(min_df=1)
print(vectorizer)

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
        tokenizer=None, vocabulary=None)


In [3]:
content = ["How to format my hard disk", "Hard disk format problems"]
X = vectorizer.fit_transform(content)
vectorizer.get_feature_names()

['disk', 'format', 'hard', 'how', 'my', 'problems', 'to']

The vectorizer has detected seven words for which we can fetch the counts individually.

In [4]:
print(X.toarray().transpose())

[[1 1]
 [1 1]
 [1 1]
 [1 0]
 [1 0]
 [0 1]
 [1 0]]


This means that the first sentence contains all the words except for "problems", while the second contains all except "how", "my", and "to". In fact, these are exactly the same columns as seen in the previous table. From X, we can extract a feature vector that we can use to compare the two documents with each other.


## Counting words

In [3]:
import os
DIR = 'data/toy'
posts = [open(os.path.join(DIR, f)).read() for f in os.listdir(DIR)]
vectorizer = CountVectorizer(min_df=1)

In [6]:
X_train = vectorizer.fit_transform(posts)
num_samples, num_features = X_train.shape
print("#samples: %d, #features: %d" %(num_samples, num_features))

#samples: 5, #features: 25


Unsurprisingle, we have five posts with a total of 25 different words. The following words that have been tokenized will be counted.

In [11]:
print(vectorizer.get_feature_names())

['about', 'actually', 'capabilities', 'contains', 'data', 'databases', 'images', 'imaging', 'interesting', 'is', 'it', 'learning', 'machine', 'most', 'much', 'not', 'permanently', 'post', 'provide', 'save', 'storage', 'store', 'stuff', 'this', 'toy']


Now we can vectorize our new post as follows:

In [7]:
new_post = "imaging databases"
new_post_vec = vectorizer.transform([new_post])
print(new_post_vec)

  (0, 5)	1
  (0, 7)	1


In [8]:
print(new_post_vec.toarray())

[[0 0 0 0 0 1 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]]


We need to use the full array if we want to use it as a vector for similarity calculations. For the similarity measurement (the naive one), we calculate the Euclidean distance between the count vectors of the new post and all the old posts as follows:

In [9]:
import scipy as sp
def dist_raw(v1, v2):
    delta = v1-v2
    return sp.linalg.norm(delta.toarray())

The norm() function calculates the Euclidean norm (shortest distance). With dist_raw, we just need to iterate over all the posts and remember the nearest one:

In [10]:
import sys
best_doc = None
best_dist = sys.maxsize
best_i = None
for i in range(0, num_samples):
    post = posts[i]
    if post == new_post:
        continue
    post_vec = X_train.getrow(i)
    d = dist_raw(post_vec, new_post_vec)
    print("=== Post %i with dist=%.2f: %s" %(i, d, post))
    if d < best_dist:
        best_dist = d
        best_i = i
print("Best post is %i with dist=%.2f" %(best_i, best_dist))

=== Post 0 with dist=1.73: Imaging databases provide storage capabilities.
=== Post 1 with dist=4.00: This is a toy post about machine learning. Actually, it contains not much interesting stuff.
=== Post 2 with dist=5.10: Imaging databases store data. Imaging databases store data. Imaging databases store data.
=== Post 3 with dist=1.41: Imaging databases store data.
=== Post 4 with dist=2.00: Most imaging databases save images permanently.

Best post is 3 with dist=1.41


Post 1 is most dissimilar from our new post. Quite understandably, it does not have a single wird in common with the new post. We can also understand that Post 0 is very similar to the new post, but not the winner, as it contains one word more than Post 3 that is not contained in the new post.

Looking at posts 2 and 3, however, the picture is not so clear any more. Post 2 is the same as post3 duplicated three times. So, it should be of the same similarity to the new post as Post 3.
Printing the corresponding feature vector explain the reason:

In [11]:
print(X_train.getrow(3).toarray())
print(X_train.getrow(2).toarray())

[[0 0 0 0 1 1 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0]]
[[0 0 0 0 3 3 0 3 0 0 0 0 0 0 0 0 0 0 0 0 0 3 0 0 0]]


Obviously, using only the counts of the raw words is too simple. We will have to normalize the to get vector of unit length.

## Normalizing the word count vectors
We will have to extend dist_raw to calculate the vector distance, not on the raw vectors but the normalized ones instead.

In [12]:
def dist_norm(v1, v2):
    v1_normalized = v1/sp.linalg.norm(v1.toarray())
    v2_normalized = v2/sp.linalg.norm(v2.toarray())
    delta = v1_normalized - v2_normalized
    return sp.linalg.norm(delta.toarray())

In [13]:
import sys
best_doc = None
best_dist = sys.maxsize
best_i = None
for i in range(0, num_samples):
    post = posts[i]
    if post == new_post:
        continue
    post_vec = X_train.getrow(i)
    d = dist_norm(post_vec, new_post_vec)
    print("=== Post %i with dist=%.2f: %s" %(i, d, post))
    if d < best_dist:
        best_dist = d
        best_i = i
print("Best post is %i with dist=%.2f" %(best_i, best_dist))

=== Post 0 with dist=0.86: Imaging databases provide storage capabilities.
=== Post 1 with dist=1.41: This is a toy post about machine learning. Actually, it contains not much interesting stuff.
=== Post 2 with dist=0.77: Imaging databases store data. Imaging databases store data. Imaging databases store data.
=== Post 3 with dist=0.77: Imaging databases store data.
=== Post 4 with dist=0.92: Most imaging databases save images permanently.

Best post is 2 with dist=0.77


This looks a bit better now. 

## Removing less important words
The best option would be to remove all words that are so frequent that they do not help to distinguish between different texts. These words are called stop words.

As this is such a common step in text processing, there is a simple parameter in CountVectorizer to achieve this, as follows:

In [15]:
vectorizer = CountVectorizer(min_df=1, stop_words='english')

If you have a clear picture of what kind of stop words you would want to remove, you can also pass a list of them. Setting stop_words to "english" will use a set of 318 English stop words. To find out which ones they are, you can use get_stop_words():

In [16]:
sorted(vectorizer.get_stop_words())[0:20]

['a',
 'about',
 'above',
 'across',
 'after',
 'afterwards',
 'again',
 'against',
 'all',
 'almost',
 'alone',
 'along',
 'already',
 'also',
 'although',
 'always',
 'am',
 'among',
 'amongst',
 'amoungst']

In [17]:
import sys
best_doc = None
best_dist = sys.maxsize
best_i = None
for i in range(0, num_samples):
    post = posts[i]
    if post == new_post:
        continue
    post_vec = X_train.getrow(i)
    d = dist_norm(post_vec, new_post_vec)
    print("=== Post %i with dist=%.2f: %s" %(i, d, post))
    if d < best_dist:
        best_dist = d
        best_i = i
print("Best post is %i with dist=%.2f" %(best_i, best_dist))

=== Post 0 with dist=0.86: Imaging databases provide storage capabilities.
=== Post 1 with dist=1.41: This is a toy post about machine learning. Actually, it contains not much interesting stuff.
=== Post 2 with dist=0.77: Imaging databases store data. Imaging databases store data. Imaging databases store data.
=== Post 3 with dist=0.77: Imaging databases store data.
=== Post 4 with dist=0.92: Most imaging databases save images permanently.

Best post is 2 with dist=0.77


Without stop words, we arruve at the following similarity measurement.

## Stemming

One thing is still missing. We count similar words in different variants as different words. Like:
* imaging and images

It would make sense to count them together. After all, it is the same concept they are referring to.
We need a function that reduces words to their specific word stem. Scikit does not contain a stemmer by default. With NLTK, wich provides a stemmer that we can easily plug into CountVectorizer.

In [18]:
import nltk.stem
s = nltk.stem.SnowballStemmer('english')
s.stem('graphics')

'graphic'

In [2]:
s.stem('imaging')

'imag'

In [3]:
s.stem('image')

'imag'

In [4]:
s.stem('imagination')

'imagin'

In [5]:
s.stem('imagine')

'imagin'

In [7]:
print(s.stem('buys'))
print(s.stem('buying'))
print(s.stem('bought'))

buy
buy
bought


## Extending the vectorizer with NLTK's Stemmer

We need to stem the posts before we feed them into CountVectorizer. The class provides several hooks with which we could customize the preprocessing and tokenization stages. The preprocessor and tokenizer can be set in the constructor as parameters. We do not want to place the stemmer into any of them, because we would then have to do the tokenization and normalization by ourselves. Instead, we overwrite the method build_analyzer as follows: 

In [1]:
import nltk
english_stemmer = nltk.stem.SnowballStemmer('english')
class StemmedCountVectorizer(CountVectorizer):
    
    def build_analyzer(self):
        analyzer = super(StemmedCountVectorizer, self).build_analyzer()
        return lambda doc: (english_stemmer.stem(w) for w in analyzer(doc))

vectorizer = StemmedCountVectorizer(min_df=1,)

NameError: name 'CountVectorizer' is not defined